In [19]:
# load document
from langchain.document_loaders import PyPDFLoader,TextLoader
loader = TextLoader("pres.txt")
documents = loader.load()

In [20]:
from langchain.chains import RetrievalQA,ConversationalRetrievalChain
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import CohereEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import Cohere

# split the documents into chunks
text_splitter = CharacterTextSplitter(chunk_size=5000, chunk_overlap=100)
texts = text_splitter.split_documents(documents)
# select which embeddings we want to use
# embeddings = CohereEmbeddings(
#     model='embed-multilingual-v2.0', cohere_api_key="9XuV53KZgETnAQXTZuJG8DsQlWaifuF1uZw6EfcO"
# )
embeddings = CohereEmbeddings(model = "multilingual-22-12",cohere_api_key="9XuV53KZgETnAQXTZuJG8DsQlWaifuF1uZw6EfcO")

# create the vectorestore to use as the index
db = Chroma.from_documents(texts, embeddings)
# expose this index in a retriever interface
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":2})
# create a chain to answer questions 
llm = Cohere(cohere_api_key="9XuV53KZgETnAQXTZuJG8DsQlWaifuF1uZw6EfcO",truncate='END')
qa = RetrievalQA.from_chain_type(
    llm=Cohere(model="command-nightly", temperature=0,cohere_api_key="9XuV53KZgETnAQXTZuJG8DsQlWaifuF1uZw6EfcO",truncate='END'), chain_type="stuff", retriever=retriever, return_source_documents=False)
query = "राष्ट्रपति का कार्य क्या है?"
result = qa({"query": query})

Number of requested results 2 is greater than number of elements in index 1, updating n_results = 1


In [21]:
result

{'query': 'राष्ट्रपति का कार्य क्या है?',
 'result': ' The answer is The President of India is the head of state and the commander-in-chief of the Indian Armed Forces.'}

In [16]:
db['embeddings']

TypeError: 'Chroma' object is not subscriptable

In [22]:
texts

[Document(page_content='शीर्षक: भारत के राष्ट्रपति: नेतृत्व के लिए एक संवैधानिक मार्गदर्शिका\n\nपरिचय:\nभारत के राष्ट्रपति का देश के लोकतांत्रिक ढांचे में सर्वोपरि महत्व का पद है। राज्य के प्रमुख के रूप में, राष्ट्रपति भारतीय संविधान के आदर्शों का प्रतीक हैं और न्याय, अखंडता और एकता के मूल्यों को बनाए रखने में महत्वपूर्ण भूमिका निभाते हैं। यह निबंध भारत के राष्ट्रपति से संबंधित संवैधानिक प्रावधानों की पड़ताल करता है और देश के शासन में इस प्रतिष्ठित कार्यालय के महत्व पर प्रकाश डालता है।\n\nभूमिका और कार्य:\nभारतीय संविधान के अनुसार, राष्ट्रपति देश की एकता का प्रतिनिधित्व करते हुए, राज्य के औपचारिक प्रमुख के रूप में कार्य करता है। अनुच्छेद 52 राष्ट्रपति को भारत के प्रथम नागरिक के रूप में स्थापित करता है और उन्हें संघ की कार्यकारी शक्ति प्रदान करता है। हालाँकि, यह ध्यान रखना महत्वपूर्ण है कि राष्ट्रपति की शक्तियाँ काफी हद तक नाममात्र की होती हैं, वास्तविक कार्यकारी अधिकार प्रधान मंत्री के नेतृत्व वाली मंत्रिपरिषद के पास होता है।\n\nचुनाव और कार्यालय का कार्यकाल:\nअनुच्छेद 54 राष्ट्रपति के

In [18]:
query

'राष्ट्रपति का कार्य क्या है?'